In [101]:
import pandas as pd
import psycopg2


# Load the uploaded CSV files
aaa_data = pd.read_csv("D:\\Github Mikezxc\\final\\Data_stock\\AAA Historical Data.csv")
aapl_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\AAPL Historical Data.csv')
acb_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\ACB Historical Data.csv')
bid_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\BID Historical Data.csv')
ctg_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\CTG Historical Data.csv')
fpt_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\FPT Historical Data.csv')
gas_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\AAPL Historical Data.csv')
nvda_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\NVDA Historical Data.csv')
vcb_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\VCB Historical Data.csv')
vnm_data = pd.read_csv('D:\\Github Mikezxc\\final\\Data_stock\\VNM Historical Data.csv')

# Display the first few rows of each dataframe to understand the structure and columns
aaa_data.head(), aapl_data.head()


(         Date     Price      Open      High       Low   Vol. Change %
 0  06/07/2024  11,850.0  11,950.0  12,100.0  11,800.0  3.87M   -0.84%
 1  06/06/2024  11,950.0  11,850.0  12,150.0  11,750.0  6.83M    1.70%
 2  06/05/2024  11,750.0  12,150.0  12,150.0  11,750.0  7.21M   -2.08%
 3  06/04/2024  12,000.0  11,950.0  12,350.0  11,950.0  5.51M    0.84%
 4  06/03/2024  11,900.0  12,000.0  12,100.0  11,850.0  5.47M    0.42%,
          Date   Price    Open    High     Low    Vol. Change %
 0  06/06/2024  194.48  195.59  196.49  194.18  41.08M   -0.71%
 1  06/05/2024  195.87  195.40  196.90  194.88  53.02M    0.78%
 2  06/04/2024  194.35  194.63  195.32  193.03  47.47M    0.16%
 3  06/03/2024  194.03  192.90  194.99  192.52  50.08M    0.93%
 4  05/31/2024  192.25  191.44  192.57  189.91  75.16M    0.50%)

In [102]:
# Function to preprocess the data with updated handling for 'Change %' column
def preprocess_data(df, stock_id):
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Weekday'] = df['Date'].dt.day_name()
    
    df['Stock_ID'] = stock_id
    df['Vol.'] = df['Vol.'].apply(lambda x: float(str(x).replace('M', '').replace('K', '')) * 1_000_000 if 'M' in str(x) else float(str(x).replace('K', '')) * 1_000 if 'K' in str(x) else float(x))
    df['Change %'] = df['Change %'].astype(str).str.replace('%', '').astype(float)
    
    return df

# Re-process the data with the updated function
aaa_data_processed = preprocess_data(aaa_data, 1)
aapl_data_processed = preprocess_data(aapl_data, 2)
acb_data_processed = preprocess_data(acb_data, 3)
bid_data_processed = preprocess_data(bid_data, 4)
ctg_data_processed = preprocess_data(ctg_data, 5)
fpt_data_processed = preprocess_data(fpt_data, 6)
gas_data_processed = preprocess_data(gas_data, 7)
nvda_data_processed = preprocess_data(nvda_data, 8)
vcb_data_processed = preprocess_data(vcb_data, 9)
vnm_data_processed = preprocess_data(vnm_data, 10)

# Select relevant columns for each table
date_dimension_columns = ['Date', 'Year', 'Month', 'Day', 'Weekday']
stock_trading_facts_columns = ['Date', 'Stock_ID', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

# Prepare Date_Dimension table data
date_dimension_data = pd.concat([aaa_data_processed[date_dimension_columns], aapl_data_processed[date_dimension_columns], acb_data_processed[date_dimension_columns], bid_data_processed[date_dimension_columns], ctg_data_processed[date_dimension_columns], fpt_data_processed[date_dimension_columns], gas_data_processed[date_dimension_columns], nvda_data_processed[date_dimension_columns],vcb_data_processed[date_dimension_columns], vnm_data_processed[date_dimension_columns]]).drop_duplicates().reset_index(drop=True)

# Prepare Stock_Trading_Facts table data
stock_trading_facts_data = pd.concat([aaa_data_processed[stock_trading_facts_columns], acb_data_processed[stock_trading_facts_columns], bid_data_processed[stock_trading_facts_columns], ctg_data_processed[stock_trading_facts_columns], fpt_data_processed[stock_trading_facts_columns], nvda_data_processed[stock_trading_facts_columns], vcb_data_processed[stock_trading_facts_columns], vnm_data_processed[stock_trading_facts_columns]]).reset_index(drop=True)
exception_fact = pd.concat([aapl_data_processed[stock_trading_facts_columns],gas_data_processed[stock_trading_facts_columns]]) 
# Prepare Stock_Dimension table data
stock_dimension_data = pd.DataFrame({
    'Stock_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Stock_Name': ['AAA', 'AAPL', 'ACB', 'BID', 'CTG', 'FPT', 'GAS', 'NVDA', 'VCB', 'VNM']
})

# Display the first few rows of each table data
date_dimension_data.head()



,Date,Year,Month,Day,Weekday
0,2024-06-07,2024,6,7,Friday
1,2024-06-06,2024,6,6,Thursday
2,2024-06-05,2024,6,5,Wednesday
3,2024-06-04,2024,6,4,Tuesday
4,2024-06-03,2024,6,3,Monday


In [103]:
stock_trading_facts_data.head()

,Date,Stock_ID,Price,Open,High,Low,Vol.,Change %
0,2024-06-07,1,"11,850.0","11,950.0","12,100.0","11,800.0",3870000.0,-0.84
1,2024-06-06,1,"11,950.0","11,850.0","12,150.0","11,750.0",6830000.0,1.70
2,2024-06-05,1,"11,750.0","12,150.0","12,150.0","11,750.0",7210000.0,-2.08
3,2024-06-04,1,"12,000.0","11,950.0","12,350.0","11,950.0",5510000.0,0.84
4,2024-06-03,1,"11,900.0","12,000.0","12,100.0","11,850.0",5470000.0,0.42


In [104]:
stock_dimension_data.head()

,Stock_ID,Stock_Name
0,1,AAA
1,2,AAPL
2,3,ACB
3,4,BID
4,5,CTG


In [105]:
# Remove commas and convert to float64
stock_trading_facts_data['Price'] = stock_trading_facts_data['Price'].str.replace(',', '').astype(float)
stock_trading_facts_data['Open'] = stock_trading_facts_data['Open'].str.replace(',', '').astype(float)
stock_trading_facts_data['High'] = stock_trading_facts_data['High'].str.replace(',', '').astype(float)
stock_trading_facts_data['Low'] = stock_trading_facts_data['Low'].str.replace(',', '').astype(float)

exception_fact.dtypes

# exception_fact['Price'] = exception_fact['Price'].astype(float)
# exception_fact['Open'] = exception_fact['Open'].astype(float)
# exception_fact['High'] = exception_fact['High'].astype(float)
# exception_fact['Low'] = exception_fact['Low'].astype(float)
# Check the data types again
stock_trading_facts_data.dtypes
# stock_trading_facts_data.to_csv('test.csv', index=False)

Date        datetime64[ns]
Stock_ID             int64
Price              float64
Open               float64
High               float64
Low                float64
Vol.               float64
Change %           float64
dtype: object

In [108]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Thông tin kết nối cơ sở dữ liệu
db_user = 'postgres'
db_password = '123'
db_host = 'localhost'
db_port = '5432'
db_name = 'thesis_stock'

# Tạo chuỗi kết nối
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Lệnh SQL để tạo schema
create_schema_sql = "CREATE SCHEMA IF NOT EXISTS stock;"

# Kết nối tới cơ sở dữ liệu PostgreSQL và tạo schema nếu chưa tồn tại
try:
    connection = psycopg2.connect(connection_string)
    cursor = connection.cursor()
    cursor.execute(create_schema_sql)
    connection.commit()
    print("Schema đã được tạo thành công")
except Exception as e:
    print(f"Lỗi khi kết nối hoặc thực thi lệnh SQL: {e}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Đã đóng kết nối đến cơ sở dữ liệu PostgreSQL")

# Đổi tên cột để chắc chắn khớp với tên mong muốn
stock_trading_facts_data.rename(columns={
    'Vol.': 'Volume',
    'Change %': 'Change_Percentage'
}, inplace=True)

exception_fact.rename(columns={
    'Vol.': 'Volume',
    'Change %': 'Change_Percentage'
}, inplace=True)

# Xử lý các giá trị không xác định (NA hoặc inf) cho DataFrame stock_trading_facts_data
stock_trading_facts_data.fillna({'Volume': 0}, inplace=True)
stock_trading_facts_data.replace({'Volume': {float('inf'): 0, -float('inf'): 0}}, inplace=True)

# Xử lý các giá trị không xác định (NA hoặc inf) cho DataFrame exception_fact
exception_fact.fillna({'Volume': 0}, inplace=True)
exception_fact.replace({'Volume': {float('inf'): 0, -float('inf'): 0}}, inplace=True)

# Đảm bảo rằng các cột có kiểu dữ liệu chính xác
Date_Dimension_data = date_dimension_data.astype({
    'Date': 'datetime64[ns]',
    'Year': 'int',
    'Month': 'int',
    'Day': 'int',
    'Weekday': 'object'
})

Stock_Dimension_data = stock_dimension_data.astype({
    'Stock_ID': 'int',
    'Stock_Name': 'object'
})

Stock_Trading_Facts_data = stock_trading_facts_data.astype({
    'Date': 'datetime64[ns]',
    'Stock_ID': 'int',
    'Price': 'float',
    'Open': 'float',
    'High': 'float',
    'Low': 'float',
    'Volume': 'int',
    'Change_Percentage': 'float'
})

Exception_Stock_Trading_Facts_data = exception_fact.astype({
    'Date': 'datetime64[ns]',
    'Stock_ID': 'int',
    'Price': 'float',
    'Open': 'float',
    'High': 'float',
    'Low': 'float',
    'Volume': 'int',
    'Change_Percentage': 'float'
})

# Tạo kết nối với SQLAlchemy
engine = create_engine(connection_string)

# Chèn dữ liệu vào các bảng trong schema 'stock'
Date_Dimension_data.to_sql('Dim_Date', engine, schema='stock', if_exists='append', index=False)
Stock_Dimension_data.to_sql('Dim_Stock', engine, schema='stock', if_exists='append', index=False)
Stock_Trading_Facts_data.to_sql('Fact_Stock', engine, schema='stock', if_exists='append', index=False)
Exception_Stock_Trading_Facts_data.to_sql('Fact_Stock', engine, schema='stock', if_exists='append', index=False)

print("Dữ liệu đã được chèn vào cơ sở dữ liệu thành công")


Schema đã được tạo thành công
Đã đóng kết nối đến cơ sở dữ liệu PostgreSQL
Dữ liệu đã được chèn vào cơ sở dữ liệu thành công
